In [143]:
sc.master
import re
from operator import add

In [144]:
type(sc)

pyspark.context.SparkContext

In [187]:
reviewsDF = spark.read.csv("gs://pda-yelp-bucket/input/review/review.csv",header='true')
reviewsDF.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+----+--------------------+
|           review_id|             user_id|         business_id|stars|      date|                text|useful|funny|cool|year|       business_name|
+--------------------+--------------------+--------------------+-----+----------+--------------------+------+-----+----+----+--------------------+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|    5|2016-05-28|Super simple plac...|     0|    0|   0|2016|          Wilensky's|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|    5|2016-05-28|Small unassuming ...|     0|    0|   0|2016|           Tuck Shop|
|MV3CcKScW05u5LVfF...|bv2nCi5Qv5vroFiqK...|CKC0-MOWMqoeWf6s-...|    5|2016-05-28|Lester s is locat...|     0|    0|   0|2016|       Lester's Deli|
|IXvOzsEMYtiJI0CAR...|bv2nCi5Qv5vroFiqK...|ACFtxLv8pGrrxMm6E...|    4|2016-05-28|Love coming here ...|     0|    0|   

In [188]:
reviewsDF = reviewsDF.select('review_id', 'business_id', 'business_name', 'stars', 'date', 'text')
reviewsDF.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+
|           review_id|         business_id|       business_name|stars|      date|                text|
+--------------------+--------------------+--------------------+-----+----------+--------------------+
|vkVSCC7xljjrAI4UG...|AEx2SYEUJmTxVVB18...|          Wilensky's|    5|2016-05-28|Super simple plac...|
|n6QzIUObkYshz4dz2...|VR6GpWIda3SfvPC-l...|           Tuck Shop|    5|2016-05-28|Small unassuming ...|
|MV3CcKScW05u5LVfF...|CKC0-MOWMqoeWf6s-...|       Lester's Deli|    5|2016-05-28|Lester s is locat...|
|IXvOzsEMYtiJI0CAR...|ACFtxLv8pGrrxMm6E...|           Five Guys|    4|2016-05-28|Love coming here ...|
|L_9BTb55X0GDtThi6...|s2I_Ni76bjJNK9yG6...|Maison Christian ...|    4|2016-05-28|Had their chocola...|
+--------------------+--------------------+--------------------+-----+----------+--------------------+
only showing top 5 rows



In [189]:
reviewsDF.select('business_id').count()

631867

In [147]:
from pyspark.sql import SQLContext

In [148]:
from pyspark.sql.types import *

In [149]:
sqlContext = SQLContext(sc)

In [150]:
from pyspark.sql import functions as fn

In [151]:
!wget https://github.com/daniel-acuna/python_data_science_intro/blob/master/data/sentiments.parquet.zip?raw=true -O sentiments.parquet.zip && unzip sentiments.parquet.zip && rm sentiments.parquet.zip

--2019-04-26 00:02:41--  https://github.com/daniel-acuna/python_data_science_intro/blob/master/data/sentiments.parquet.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/daniel-acuna/python_data_science_intro/raw/master/data/sentiments.parquet.zip [following]
--2019-04-26 00:02:41--  https://github.com/daniel-acuna/python_data_science_intro/raw/master/data/sentiments.parquet.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/daniel-acuna/python_data_science_intro/master/data/sentiments.parquet.zip [following]
--2019-04-26 00:02:41--  https://raw.githubusercontent.com/daniel-acuna/python_data_science_intro/master/data/sentiments.parquet.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.

In [153]:
sentiments_df = sqlContext.read.parquet('hdfs:///sentiments.parquet')

In [154]:
sentiments_df.columns

['word', 'sentiment']

In [155]:
from pyspark.ml.feature import RegexTokenizer

In [156]:
tokenizer = RegexTokenizer().setGaps(False)\
  .setPattern("\\p{L}+")\
  .setInputCol("text")\
  .setOutputCol("words")

In [190]:
reviews_words_df = tokenizer.transform(reviewsDF)
print(reviews_words_df)

DataFrame[review_id: string, business_id: string, business_name: string, stars: string, date: string, text: string, words: array<string>]


In [191]:
reviews_words_df.show(5)

+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|           review_id|         business_id|       business_name|stars|      date|                text|               words|
+--------------------+--------------------+--------------------+-----+----------+--------------------+--------------------+
|vkVSCC7xljjrAI4UG...|AEx2SYEUJmTxVVB18...|          Wilensky's|    5|2016-05-28|Super simple plac...|[super, simple, p...|
|n6QzIUObkYshz4dz2...|VR6GpWIda3SfvPC-l...|           Tuck Shop|    5|2016-05-28|Small unassuming ...|[small, unassumin...|
|MV3CcKScW05u5LVfF...|CKC0-MOWMqoeWf6s-...|       Lester's Deli|    5|2016-05-28|Lester s is locat...|[lester, s, is, l...|
|IXvOzsEMYtiJI0CAR...|ACFtxLv8pGrrxMm6E...|           Five Guys|    4|2016-05-28|Love coming here ...|[love, coming, he...|
|L_9BTb55X0GDtThi6...|s2I_Ni76bjJNK9yG6...|Maison Christian ...|    4|2016-05-28|Had their chocola...|[had, their, choc...|
+-------

In [192]:
reviews_word_sentiment_df = reviews_words_df.\
    select('review_id', fn.explode('words').alias('word')).\
    join(sentiments_df, 'word')
reviews_word_sentiment_df.show(5)

+--------+--------------------+---------+
|    word|           review_id|sentiment|
+--------+--------------------+---------+
|   super|vkVSCC7xljjrAI4UG...|        1|
| amazing|vkVSCC7xljjrAI4UG...|        1|
| helpful|vkVSCC7xljjrAI4UG...|        1|
|friendly|vkVSCC7xljjrAI4UG...|        1|
|    cool|n6QzIUObkYshz4dz2...|        1|
+--------+--------------------+---------+
only showing top 5 rows



In [193]:
sentiment_prediction_df = reviews_word_sentiment_df.\
    groupBy('review_id').\
    agg(fn.avg('sentiment').alias('avg_sentiment'))
sentiment_prediction_df.show(5)

+--------------------+------------------+
|           review_id|     avg_sentiment|
+--------------------+------------------+
|eaqMSHRiCGNvCTIZJ...|               1.0|
|tYR1xyPllaotWTVtf...|               1.0|
|qGAuB1RR5YsCuyoNX...|               0.5|
|TvmJUP92QlsTZ-JW7...|               0.8|
|BBGhTeVpUIB9uqd0F...|0.5555555555555556|
+--------------------+------------------+
only showing top 5 rows



In [194]:
reviews_sentiment_df = sentiment_prediction_df.\
    join(reviewsDF, 'review_id')
    
reviews_sentiment_df.show(5)

+--------------------+-------------------+--------------------+--------------------+-----+----------+--------------------+
|           review_id|      avg_sentiment|         business_id|       business_name|stars|      date|                text|
+--------------------+-------------------+--------------------+--------------------+-----+----------+--------------------+
|--Jud58FsHHomDBai...| 0.2727272727272727|OgJ0KxwJcJ9R5bUK0...|         Lux Central|    5|2016-04-10|I m pretty overdu...|
|-01ePuKPxMw1bhR4I...| 0.8333333333333334|24Td_CQH1bonWKff1...|Matt's Big Breakfast|    4|2016-05-19|Been eating at Ma...|
|-0u6BAh47_WiKXcjY...| 0.7272727272727273|acgqti4AEuMIFUYss...|         Cafe Belong|    3|2016-12-12|3 5 stars        ...|
|-1kQanNhit-7B9RBj...|0.06666666666666667|41Kbugm-qmIQTt7un...|Top of Binion's S...|    3|2016-06-01|The great  epic g...|
|-2WBhKLf7NU0DffKY...|                1.0|Ec9CBmL3285XkeHaN...|        District One|    4|2016-06-19|Great food  servi...|
+---------------

In [196]:
businessDF = spark.read.csv("gs://pda-yelp-bucket/input/business/business.csv",header='true')
businessDF = businessDF.select('business_id', 'city', 'state','latitude', 'longitude')
businessDF.show(5)

+--------------------+--------------+-----+----------+------------+
|         business_id|          city|state|  latitude|   longitude|
+--------------------+--------------+-----+----------+------------+
|FYWN1wneV18bWNgQj...|     Ahwatukee|   AZ|33.3306902|-111.9785992|
|He-G7vWjzVUysIKrf...|      McMurray|   PA|40.2916853| -80.1048999|
|KQPW8lFf1y5BT2Mxi...|       Phoenix|   AZ|33.5249025|-112.1153098|
|8DShNS-LuFqpEWIp0...|         Tempe|   AZ|33.3831468|-111.9647254|
|PfOCPjBrlQAnz__NX...|Cuyahoga Falls|   OH|41.1195346| -81.4756898|
+--------------------+--------------+-----+----------+------------+
only showing top 5 rows



In [197]:
reviews_business = reviews_sentiment_df.\
    join(businessDF, 'business_id')    
reviews_business.show(5)

+--------------------+--------------------+------------------+--------------------+-----+----------+--------------------+---------+-----+---------+----------+
|         business_id|           review_id|     avg_sentiment|       business_name|stars|      date|                text|     city|state| latitude| longitude|
+--------------------+--------------------+------------------+--------------------+-----+----------+--------------------+---------+-----+---------+----------+
|--9e1ONYQuAa-CB_R...|7aj1xcFBYiH1Jg_OX...|              0.25|Delmonico Steakhouse|    1|2016-05-22|Ohhh where to bei...|Las Vegas|   NV|36.123183|-115.16919|
|--9e1ONYQuAa-CB_R...|7dVUVdmWIIXQ5KtF_...|              0.75|Delmonico Steakhouse|    5|2016-11-26|This is my third ...|Las Vegas|   NV|36.123183|-115.16919|
|--9e1ONYQuAa-CB_R...|UxFpgng8dPMWOj996...|               1.0|Delmonico Steakhouse|    5|2016-02-10|Truly Fantastic  ...|Las Vegas|   NV|36.123183|-115.16919|
|--9e1ONYQuAa-CB_R...|i7xD3FY-EaF9O08QL...|   

In [206]:
reviews_business.columns

['business_id',
 'review_id',
 'avg_sentiment',
 'business_name',
 'stars',
 'date',
 'text',
 'city',
 'state',
 'latitude',
 'longitude']

In [211]:
reviews_sentiment_score_df =  reviews_business.\
    groupBy('business_name', 'city').\
    agg(fn.mean('avg_sentiment').alias('avg_sentiment'))
reviews_sentiment_score_df.show(5)

+--------------------+--------------+------------------+
|       business_name|          city|     avg_sentiment|
+--------------------+--------------+------------------+
|        John's Diner|      Lakewood|0.5714285714285714|
|Lo-Lo's Chicken a...|       Gilbert|0.4166322779619457|
|         Chick-fil-A|Cuyahoga Falls|0.7999999999999999|
|        Yala's Pizza|        Lorain|0.6296296296296297|
|   The Flame Broiler|     Las Vegas| 0.761476370851371|
+--------------------+--------------+------------------+
only showing top 5 rows



In [215]:
reviews_avg_rating_df =  reviews_business.\
    groupBy('business_name', 'city').\
    agg(fn.mean('stars').alias('rating'))
reviews_avg_rating_df.show(5)

+--------------------+--------------+------------------+
|       business_name|          city|            rating|
+--------------------+--------------+------------------+
|        John's Diner|      Lakewood|               3.5|
|Lo-Lo's Chicken a...|       Gilbert|3.2479338842975207|
|         Chick-fil-A|Cuyahoga Falls| 4.333333333333333|
|        Yala's Pizza|        Lorain|               4.0|
|   The Flame Broiler|     Las Vegas|               4.5|
+--------------------+--------------+------------------+
only showing top 5 rows



In [218]:
reviews_avg_rating_df = reviews_avg_rating_df.select('business_name', 'rating')
sentiment_rating = reviews_sentiment_score_df.\
    join(reviews_avg_rating_df, 'business_name')
sentiment_rating.show(5)

+--------------------+-------------+------------------+------------------+
|       business_name|         city|     avg_sentiment|            rating|
+--------------------+-------------+------------------+------------------+
|        Adobo Dragon|      Phoenix|0.6154109309189524|4.0606060606060606|
|Ahnala Mesquite Room|Fort McDowell|0.5095238095238096|               3.6|
|     Ahuja Cafeteria|    Beachwood|0.3333333333333333|               1.5|
|  Allegro Fine Foods|       Bolton|0.7222222222222222| 4.666666666666667|
|       Allo Mon Coco|Pointe-Claire|0.4857142857142857|               4.5|
+--------------------+-------------+------------------+------------------+
only showing top 5 rows



In [220]:
outputpath="gs://pda-yelp-bucket/output/sentimentScore/"
sentiment_rating.coalesce(1).write.save(outputpath,sep=",",mode="append",format="csv",header="TRUE")
